# Deep Learning & Neural Networks
## Project 3 - Handwriting Recognition with ConvNets

### SETUP

In [ ]:
# Essentially
# https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html
# but with inline comments

# Load TensorFlow
import tensorflow as tf
# Load numpy - adds MATLAB/Julia-style math to Python
import numpy as np
# Load matplotlib for plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Load MNIST data set
# Will download into working directory if not already there
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
# Some helper functions
# Create, initialize, and return a weight matrix
# of the provided shape
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
# Ditto, but for a bias term
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
# TF lets you get fancy with convolutions
# We want generic basic vanilla ones:
# Stride: 1
# Padding (for edges): use 0s
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# Similar for our pooling: we'll use simple 2x2 max pooling
# with no overlaps - so the output will be half the height
# and width of the input
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')

### LIVE CODING BEGINS

In [ ]:
sess = tf.Session()

# Define our inputs
x  = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

# The images are already as vectors
# Which is normally good!
# But convolutions actually work on the matrix
# Lets use the reshape operator to mangle them
# into matrices.
x_image = tf.reshape(x, [-1,28,28,1])
# Thats an unknown number of 28x28 images with 1 channel

# First convolutional layer
# 5x5 filters, 1 input channel, 32 kinds of filter
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
# We'll use ReLUs
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# And our basic max pool
h_pool1 = max_pool_2x2(h_conv1)
# Now we have something that is ? x 14 x 14 x 32

# Second convolutional layer
# Again 5x5 filters, but now 64 of them
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
# Again use ReLUs and 2x2 maxpool
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Now we have 7x7, with 64 channels
# 7x7x64 = 3136
# We'll have one "fully connected" ReLU layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
# Mangle back to vector
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# ReLU it
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# New trick: dropout
# Switch to slides to explain
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Now we need an output layer
# We want a 0-1 value for each of the 10 digits, so
# lets go with softmax
W_fc2  = weight_variable([1024, 10])
b_fc2  = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### LIVE CODING ENDS

In [ ]:
# Time for the training!
# We'll use a "cross entropy" loss function instead
# of square loss
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
# We'll use ADAM instead of SGD (fancy!)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# We can use TF to track the accuracy
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
# Initialize weights
sess.run(tf.initialize_all_variables())
# Time to train this thing
# Warning, may melt laptop
ces, accs = [], []
for i in range(1000):
    # Use the helper functions to get a batch of
    # 50 digits
    batch = mnist.train.next_batch(50)
    # Every 100 steps
    if i%100 == 0:
        # Accuracy is measured with dropout off
        train_accuracy = sess.run(accuracy, 
                                  feed_dict={x:batch[0],
                                             y_: batch[1],
                                             keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    # Train it
    ce, acc, _ = sess.run((cross_entropy,accuracy,train_step),
             feed_dict={x: batch[0],
                        y_: batch[1],
                        keep_prob: 0.5})
    ces.append(ce)
    accs.append(acc)

In [ ]:
f, axarr = plt.subplots(2, sharex=True)
axarr[0].plot(range(1000), ces)
axarr[1].plot(range(1000), accs)
print("test accuracy %g"%sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
# Lets have a look at one shall we?
idx = 10
plt.matshow(np.reshape(mnist.test.images[idx], (28,28)))
net_opinion = sess.run(y_conv, feed_dict={
    x: mnist.test.images[idx:idx+1], keep_prob: 1.0})
print np.round(net_opinion)
print np.argmax(net_opinion)

In [ ]:
# Can we find a mistake?
for idx in range(300, 1000):
    net_opinion = sess.run(y_conv, feed_dict={
        x: mnist.test.images[idx:idx+1], keep_prob: 1.0})
    net_digit = np.argmax(net_opinion)
    true_digit = np.argmax(mnist.test.labels[idx])
    if net_digit != true_digit:
        plt.matshow(np.reshape(mnist.test.images[idx], (28,28)))
        print idx
        print net_opinion
        print net_digit
        print true_digit
        break

In [ ]:
# Lets look at the activations in the first convolutional layer
idx = 10
plt.matshow(np.reshape(mnist.test.images[idx], (28,28)))
value_h_conv1 = sess.run(h_conv1, feed_dict={
    x: mnist.test.images[idx:idx+1], keep_prob: 1.0})
plt.matshow(value_h_conv1[0,:,:,0])
plt.matshow(value_h_conv1[0,:,:,1])